In [1]:
import pandas as pd
import requests
import json
import inspect
%pip install quandl
%pip install fredapi
%pip install DBnomics
import quandl
quandl.ApiConfig.api_key = "P8E9VjuyG6eJfHve8-rU"
import datetime as dt
import time
import csv
import os

import inspect

os.chdir("C:/Users/marti/Dropbox/python_work/")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [113]:
## Get economic data

cpi_china = pd.DataFrame(quandl.get("NBSC/A01010101_M") - 100)
cpi_china.columns = ["cpi_china_yoy"]
cpi_china.rename_axis("date", axis='index', inplace=True)

cpi_usa = pd.DataFrame(quandl.get("FRED/SUUR0000SA0"))
cpi_usa = cpi_usa.pct_change(periods=12)*100
cpi_usa = cpi_usa.resample('M').last() ## Change date to be last day of the month
cpi_usa.columns = ["cpi_usa_yoy"]

# Calculate the difference between Chinese and US inflation rates
cpi_usa = cpi_usa.loc["2005-01-01" : "2020-07-31"]
cpi_china = cpi_china.loc["2005-01-01" : "2020-07-31"]

cpi_differential = pd.DataFrame()
cpi_differential["inflation_chinaminusus"] = cpi_china["cpi_china_yoy"] - cpi_usa["cpi_usa_yoy"]
cpi_differential.rename_axis("date", axis='index', inplace=True)

m2_yoy_china = pd.DataFrame(quandl.get("NBSC/A0B0102_M"))
m2_yoy_china.columns = ["m2_yoy_china"]

# USDCNY
usdcny = pd.DataFrame(quandl.get("FRED/EXCHUS"))
usdcny = usdcny.resample('M').last() ## Change date to be last day of the month
usdcny.columns = ["usdcny"]
usdcny.rename_axis("date", axis='index', inplace=True)

# US yields
usa_treasuries = pd.DataFrame(quandl.get("USTREASURY/YIELD"))
usa_treasuries = usa_treasuries.resample('M').last() ## Change date to be last day of the month
usa_treasuries.columns = usa_treasuries.columns.str.replace(' ', '').str.lower()
usa_treasuries = usa_treasuries.rename({'Date': 'date'},axis=1)

usa_10yr = pd.DataFrame()
usa_10yr["usa_10yr"] = usa_treasuries["10yr"]
usa_10yr.rename_axis("date", axis='index', inplace=True)

usa_2yr = pd.DataFrame()
usa_2yr["usa_2yr"] = usa_treasuries["2yr"]
usa_2yr.rename_axis("date", axis='index', inplace=True)

usa_yieldcurve_2vs10 = pd.DataFrame()
usa_yieldcurve_2vs10["usa_yieldcurve_2vs10"] = usa_10yr["usa_10yr"] - usa_2yr["usa_2yr"]
cpi_differential

,inflation_chinaminusus
date,
2005-01-31,-0.671845
2005-02-28,1.381747
2005-03-31,0.044527
2005-04-30,-1.265309
2005-05-31,-0.680830
...,...
2020-03-31,3.078134
2020-04-30,3.346881
2020-05-31,2.703359


In [36]:
## Get economic policy uncertainty data
china_epu_english = pd.read_excel("https://www.policyuncertainty.com/media/SCMP_China_Policy_Uncertainty_Data.xlsx")
china_epu_chinese_tpu = pd.read_excel("https://www.policyuncertainty.com/media/China_Mainland_Paper_EPU.xlsx", 'TPU 2000 onwards', dtype={'year':int, 'month': int, 'TPU': int})
china_epu_chinese_epu = pd.read_excel("https://www.policyuncertainty.com/media/China_Mainland_Paper_EPU.xlsx", 'EPU 2000 onwards', dtype={'year':int, 'month': int, 'EPU': int})

china_epu_english.drop(china_epu_english.index[310], inplace=True)
china_epu_english["day"] = "28"
china_epu_english["date"] = pd.to_datetime(china_epu_english[['year', 'month', 'day']])
from pandas.tseries.offsets import MonthEnd
china_epu_english['date'] = pd.to_datetime(china_epu_english['date'], format="%Y%m")+MonthEnd(0)
china_epu_english.drop(['year', 'month', 'day'], axis=1, inplace=True)
china_epu_english.set_index("date", inplace=True)
china_epu_english.rename(columns={china_epu_english.columns[0]: "china_epu_english"}, inplace = True)

china_epu_chinese_tpu["day"] = "28"
china_epu_chinese_tpu["date"] = pd.to_datetime(china_epu_chinese_tpu[['year', 'month', 'day']])
china_epu_chinese_tpu['date'] = pd.to_datetime(china_epu_chinese_tpu['date'], format="%Y%m")+MonthEnd(0)
china_epu_chinese_tpu.drop(['year', 'month', 'day'], axis=1, inplace=True)
china_epu_chinese_tpu.set_index("date", inplace=True)
china_epu_chinese_tpu.rename(columns={china_epu_chinese_tpu.columns[0]: "china_epu_chinese_tpu"}, inplace = True)
china_epu_chinese_tpu.columns = ['china_epu_chinese_tpu', 'column2', 'column3', 'column4'] 
china_epu_chinese_tpu.drop(["column2", "column3", "column4"],axis=1, inplace=True)


In [41]:
ceicdata = pd.read_excel("20201101 CEIC Chinese data.xlsx")
ceicdata = ceicdata.drop(ceicdata.index[0:5])
ceicdata = ceicdata.drop(ceicdata.columns[2:4], axis=1)
ceicdata= ceicdata.drop(list(ceicdata)[-14:], axis=1)
ceicdata.rename(columns={ceicdata.columns[0]: "date"}, inplace = True)
ceicdata['date'] = pd.to_datetime(ceicdata['date']).dt.date
from pandas.tseries.offsets import MonthEnd
ceicdata['date'] = pd.to_datetime(ceicdata['date'], format="%Y-%m-%d") + MonthEnd(1)
ceicdata.set_index("date")

china_2yr = ceicdata.drop(ceicdata.columns[12:], axis=1)
china_2yr = china_2yr.drop(china_2yr.columns[1:3], axis=1)
china_2yr = china_2yr.drop(china_2yr.columns[2:], axis=1)
china_2yr = china_2yr.rename({'CN: Treasury Bond Yield: Interbank: Spot Yield: 2 year': 'china_2yr'},axis=1)

china_10yr = ceicdata.drop(ceicdata.columns[12:], axis=1)
china_10yr = china_10yr.drop(china_10yr.columns[1:11], axis=1)
china_10yr = china_10yr.rename({'CN: Treasury Bond Yield: Interbank: Spot Yield: 10 year': 'china_10yr'},axis=1)

china_yieldcurve_10vs2 = china_10yr
china_yieldcurve_10vs2['china_yieldcurve_2vs10'] = china_10yr['china_10yr'] - china_2yr['china_2yr']
china_yieldcurve_10vs2 = china_yieldcurve_10vs2.drop(china_yieldcurve_10vs2.columns[1], axis=1)

china_bop_monthly = ceicdata.drop(ceicdata.columns[1:30], axis=1)
china_bop_monthly = china_bop_monthly.rename({'CN: BoP: Current Account: Goods & Service': 'china_bop_monthly'},axis=1)

In [68]:
fx_dataset = china_yieldcurve_10vs2
fx_dataset = fx_dataset.merge(usdcny, how='left', left_index=True, right_index=True)
fx_dataset = fx_dataset.merge(china_10yr, how='left')
fx_dataset = fx_dataset.merge(china_2yr, how='left')
fx_dataset = fx_dataset.merge(china_bop_monthly, how='left')
fx_dataset = fx_dataset.merge(china_yieldcurve_10vs2, how='left')
fx_dataset = fx_dataset.merge(usa_10yr, how='left', on='date')

fx_dataset.set_index("date", inplace=True, drop=True)
fx_dataset = fx_dataset.merge(china_epu_chinese_tpu, how='left', left_index=True, right_index=True)
fx_dataset = fx_dataset.merge(china_epu_english, how='left', left_index=True, right_index=True)
fx_dataset = fx_dataset.loc["2005-01-01" : "2020-06-30"]

fx_dataset

KeyError: 'date'

In [ ]:
china_tradebalance['china_tradebalance'] = china_tradebalance['china_tradebalance']/gdp_china['gdp_china']

fx_dataset = fx_dataset.merge(
fx_dataset = fx_dataset.merge(
fx_dataset = fx_dataset.merge(
fx_dataset = fx_dataset.merge(
    fx_dataset = fx_dataset.merge(china_tradebalance, how='left')




  
    

fx_dataset = fx_dataset.loc["2005-01-01" : "2020-06-30"]




In [ ]:
frames = [usdcny, cpi_china, cpi_usa, cpi_differential, exports_china, imports_china, m2_yoy_china, usa_2yr, usa_10yr, china_epu_chinese_tpu, china_epu_english]
result = pd.concat(frames)
                          
rmbdataset = []
rmbdataset = result.loc["2005-01-01" : "2020-06-30"]
print(rmbdataset)

print(rmbdataset["cpi_china_yoy"])

In [16]:

gdp_china = quandl.get("NBSC/A010101_Q")
gdp_china.columns = ["gdp_china"]
gdp_china = gdp_china.resample('M').fillna(method="ffill")
gdp_china = gdp_china.loc["2005-01-01" : "2020-06-30"]
gdp_china = gdp_china.rename_axis('date')

china_tradebalance = ceicdata.drop(ceicdata.columns[2:], axis=1)
china_tradebalance = china_tradebalance.rename(columns = {'CN: Trade Balance': 'china_tradebalance'},inplace=True)
china_tradebalance = ceicdata.drop(ceicdata.columns[2:], axis=1)
china_tradebalance = china_tradebalance.loc["2005-01-31": "2020-06-30"]
china_tradebalance = china_tradebalance['china_tradebalance'].divide(gdp_china['gdp_china'])


KeyError: 'china_tradebalance'